# Config agent

Chat to fill in config variables in State

In [1]:
from typing import Optional, List, Union, TypedDict, Annotated

import sqlite3

from datetime import date, datetime
from pydantic import BaseModel, Field

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_core.tools import tool

from langchain_google_vertexai import ChatVertexAI

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import MemorySaver

from promptgit import Prompt

In [2]:
class ConfigState(BaseModel):
    name: str = None
    street: str = None
    city: str = None
    messages: Annotated[list, add_messages] = []

In [3]:
@tool
def update_name(name: str, config: RunnableConfig):
    """ 
    Update name
    """
    return {'name': name}

@tool
def update_adress(street: str, city: str, config: RunnableConfig):
    """
    Update adress. Need both street and city
    """
    return {'street': street, 'city': city}

In [4]:
def initialize(state: ConfigState, config: RunnableConfig):
    if state.messages == []:
        return {'messages': [
            SystemMessage('You are setting user in the application. Your aim is to fill all the data in short conversation. Please provide name and adress (city and street)'), 
            HumanMessage('Hi'),
#            AIMessage('Please provide name and adress (city and street)')
        ]}
    else:
        return

In [5]:
def chatbot(state: ConfigState, config: RunnableConfig):
    return {'messages':[llm.invoke(state.messages)]}

In [6]:
# def tool_condition():
#     pass
from langgraph.prebuilt import tools_condition

In [7]:
llm = ChatVertexAI(model='gemini-1.5-flash-002').bind_tools([update_name, update_adress])

# checkpointer = SqliteSaver(sqlite3.connect('checkpoints.sqlite', check_same_thread=False))
checkpointer = MemorySaver()

In [8]:
graph_builder = StateGraph(ConfigState)

graph_builder.add_node("initialize", initialize)
graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "initialize")
graph_builder.add_edge("initialize", "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(checkpointer=checkpointer)

In [9]:
config = {"configurable": {"thread_id": "42"}}

In [10]:
response = graph.invoke({'messages':[]}, config)

In [11]:
graph.get_state(config)

StateSnapshot(values={'messages': [SystemMessage(content='You are setting user in the application. Your aim is to fill all the data in short conversation. Please provide name and adress (city and street)', additional_kwargs={}, response_metadata={}, id='bd3ac25e-081a-4e90-8b7c-a4979375ff31'), HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='2e89e612-738c-4603-8a30-bcba58964985'), AIMessage(content='Hello! What is your name?\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 53, 'candidates_token_count': 8, 'total_token_count': 61, 'cached_content_token_count': 0}, 'finish_reason': 'STOP'}, id='run-0caa0966-3a77-4e5c-8121-e8e898821e7d-0', usage_metadata={'input_tokens': 53, 'output_tokens': 8, 'total_tokens': 61})]}, next=(), config={'configurable': {'thread_id': '42', 'checkpoint_ns': '', 'checkpoint_id': '1ef92f5d-0692-6b80-8002-9a0a64cef9f5'}}, metadata={'source': 'loop', 'writes':